In [111]:
from imblearn.over_sampling import ADASYN
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import cv2
import os

In [112]:
# Load dataset
def load_images_and_labels(dataset_path, img_size=(224, 224)):
    images, labels = [], []
    for label_dir in os.listdir(dataset_path):
        for img_file in os.listdir(os.path.join(dataset_path, label_dir)):
            img_path = os.path.join(dataset_path, label_dir, img_file)
            image = cv2.imread(img_path)
            image = cv2.resize(image, img_size)
            images.append(image)
            labels.append(label_dir)
    return np.array(images), np.array(labels)


In [113]:
images, labels = load_images_and_labels("./wheat_leaf")
images = images / 255.0 

In [114]:
# Encode labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [115]:
# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [167]:
from collections import Counter

# Check class distribution in the training set
from collections import Counter
class_distribution = Counter(np.argmax(y_train, axis=1))
print("Class distribution in the training set:", class_distribution)

# Calculate the desired number of samples per class based on your strategy
majority_class_count = max(class_distribution.values())
sampling_strategy = {class_id: majority_class_count for class_id in class_distribution.keys()}

# Apply ADASYN with the sampling strategy for each class
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
adasyn = ADASYN(sampling_strategy=sampling_strategy, random_state=42, n_neighbors=5)
X_resampled, y_resampled = adasyn.fit_resample(X_train_flattened, y_train)

# Reshape back to image dimensions
X_resampled = X_resampled.reshape(-1, 224, 224, 3)


Class distribution in the training set: Counter({2: 150, 1: 70, 0: 64})


In [168]:
# Build the model using MobileNetV2
mobilenet = MobileNetV2(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
mobilenet.trainable = False

model = models.Sequential([
    mobilenet,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),  # Dense layer for feature learning
    layers.Dense(3, activation='softmax')  # Output layer for 3 classes
])

In [169]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [173]:
model.fit(X_resampled, y_resampled, epochs=20, validation_data=(X_valid, y_valid)) 

Epoch 1/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 12s 712ms/step - accuracy: 1.0000 - loss: 0.0058 - val_accuracy: 0.9180 - val_loss: 0.1427
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 20s 646ms/step - accuracy: 1.0000 - loss: 0.0045 - val_accuracy: 0.9508 - val_loss: 0.1282
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 634ms/step - accuracy: 1.0000 - loss: 0.0038 - val_accuracy: 0.9508 - val_loss: 0.1324
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 619ms/step - accuracy: 1.0000 - loss: 0.0040 - val_accuracy: 0.9508 - val_loss: 0.1285
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 10s 651ms/step - accuracy: 1.0000 - loss: 0.0034 - val_accuracy: 0.9344 - val_loss: 0.1408
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 598ms/step - accuracy: 1.0000 - loss: 0.0027 - val_accuracy: 0.9508 - val_loss: 0.1328
Epoch 7/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 574ms/step - accuracy: 1.0000 - loss: 0.0026 - val_accuracy: 0.9508 - val_loss: 0.1344
Epoch 8/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 568ms/step - accuracy: 1.0000 - loss: 0.0025 - val_accuracy:

In [174]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy:.2f}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.9362 - loss: 0.1190
Test accuracy: 0.94


In [175]:
# Generate predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 5s/step


In [176]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Confusion Matrix and Classification Report
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)

print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes, target_names=lb.classes_))


Confusion Matrix:
[[16  0  2]
 [ 0 12  1]
 [ 1  0 30]]
Classification Report:
              precision    recall  f1-score   support

     Healthy       0.94      0.89      0.91        18
    septoria       1.00      0.92      0.96        13
 stripe_rust       0.91      0.97      0.94        31

    accuracy                           0.94        62
   macro avg       0.95      0.93      0.94        62
weighted avg       0.94      0.94      0.94        62



In [177]:
# Save the model
model.save("mobilenet_v2_model.h5")

In [178]:
# Test the model with a new image
def predict_image(model, image_path, label_binarizer):
    # Load and preprocess the test image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    image = image / 255.0  # Normalize
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    
    # Predict the class
    prediction = model.predict(image)
    predicted_class = np.argmax(prediction, axis=1)
    
    # Decode the label
    decoded_label = label_binarizer.inverse_transform(prediction)
    return decoded_label[0]

# Load the saved model
from tensorflow.keras.models import load_model
model = load_model("mobilenet_v2_model.h5")

# Test with "test.jpg"
test_image_path = "test.jpg"
predicted_label = predict_image(model, test_image_path, lb)
print(f"The predicted label for the test image is: {predicted_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
The predicted label for the test image is: stripe_rust
